<a href="https://colab.research.google.com/github/jaugusto/yolo-darknet-opencv/blob/main/australian_street_traffic_object_detection(YOLO_and_opencv_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AUSTRALIAN STREET TRAFFIC OBJECT DETECTION**

In [ ]:
!pip install opencv-python==4.4.0.40

In [ ]:
import cv2
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import zipfile


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
zip_path = '/content/gdrive/MyDrive/YOLO_exemplos/YOLO/modelo_YOLOv4.zip'
zip_object = zipfile.ZipFile(file=zip_path, mode='r')
zip_object.extractall()
zip_object.close()

In [ ]:
weights_path = os.path.sep.join(['/content','yolov4.weights'])
config_path = os.path.sep.join(['/content/cfg','yolov4.cfg'])
labels_path = os.path.sep.join(['/content/cfg','coco.names'])

LABELS = open(labels_path).read().strip().split("\n")

In [ ]:
net = cv2.dnn.readNet(config_path, weights_path)

In [ ]:
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype='uint8')

In [ ]:
ln = net.getLayerNames()
ln = [ln[i[0]-1] for i in net.getUnconnectedOutLayers()]

In [ ]:
def mostrar(img):
  fig = plt.gcf()
  fig.set_size_inches(16,10)
  plt.axis('off')
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
def blob_imagem(net, imagem):
  blob = cv2.dnn.blobFromImage(imagem, 1/255.0, (416,416), swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)
  return net, imagem, layerOutputs


In [ ]:
def deteccoes(detection, threshold, caixas, confiancas, IDclasses):
  scores = detection[5:]
  classeID = np.argmax(scores)
  confianca = scores[classeID]

  if(confianca > threshold):
    caixa = detection[0:4] * np.array([W, H, W, H])
    (centerX, centerY, width, height) = caixa.astype('int')

    x = int(centerX - (width / 2))
    y = int(centerY - (height / 2))

    caixas.append([x, y, int(width), int(height)])
    confiancas.append(float(confianca))
    IDclasses.append(classeID)
  
  return caixas, confiancas, IDclasses

In [ ]:
def funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS):
  (x,y) = (caixas[i][0], caixas[i][1])
  (w,h) = (caixas[i][2], caixas[i][3])

  cor = [int(c) for c in COLORS[IDclasses[i]]]
  cv2.rectangle(imagem, (x,y), (x+w, y+h), cor, 2)
  texto = "{} : {:.4f}".format(LABELS[IDclasses[i]], confiancas[i])
  cv2.putText(imagem, texto, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)

  return imagem, x, y, w, h

In [ ]:
!cp  '/content/gdrive/MyDrive/YOLO_exemplos/australia.mp4' ./

In [ ]:
arquivo_video = 'australia.mp4'
cap = cv2.VideoCapture(arquivo_video)
conectado, video = cap.read()

video_altura = video.shape[0]
video_largura = video.shape[1]

In [ ]:
def redimensionar(largura, altura, largura_maxima=600):
  if(largura > largura_maxima):
    proporcao = largura/altura
    video_largura = largura_maxima
    video_altura = int(video_largura/proporcao)
  else:
    video_largura = largura
    video_altura = altura
  return video_largura, video_altura

video_largura, video_altura = redimensionar(video_largura, video_altura)

In [ ]:
nome_arquivo = 'australia2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 24

saida_video = cv2.VideoWriter(nome_arquivo, fourcc, fps, (video_largura, video_altura))

In [ ]:
threshold = 0.5
threshold_NMS = 0.3
fonte_pequena, fonte_media = 0.4, 0.6
fonte = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
while(cv2.waitKey(1) <0):
  conectado, frame = cap.read()
  if not conectado:
    break
  frame = cv2.resize(frame, (video_largura, video_altura))
  try:
    (H,W) = frame.shape[:2]
  except:
    print('Erro no redimensionamento')
    continue

  imagem_cp = frame.copy()
  net, frame, layerOutputs = blob_imagem(net, frame)
  caixas = []
  confiancas = []
  IDclasses = []


  for output in layerOutputs:
    for detection in output:
      caixas, confiancas, IDclasses = deteccoes(detection, threshold, caixas, confiancas, IDclasses)
  
  objs = cv2.dnn.NMSBoxes(caixas, confiancas, threshold, threshold_NMS)

  if(len(objs)>0):
    for i in objs.flatten():
      frame, x, y, w, h = funcoes_imagem(frame, i, confiancas, caixas, COLORS, LABELS)
      objeto = imagem_cp[y:y+h, x:x+w]
  
  cv2.putText(frame, "frame processado", (20, video_altura-20), fonte, fonte_pequena, (250, 250, 250), 0, lineType=cv2.LINE_AA)

  saida_video.write(frame)

saida_video.release()
cv2.destroyAllWindows()